In [1]:
from openai import OpenAI
import json
import pandas as pd

In [3]:
# Load the JSON file
with open('../data/raw_appraisals_dataset.json', 'r') as f:
    data = json.load(f)

In [4]:
subjects = pd.DataFrame([a['subject'] for a in data['appraisals']])
print(subjects.head())

                                   address    subject_city_province_zip  \
0  142-950 Oakview Ave Kingston ON K7M 6W8           "Twin Oak Meadows"   
1           7180 207 HWY Halifax NS B0J2L0  West Chezzetcook, NS B0J2L0   
2                11 PAUL AVE Ayr ON N0B1E0                Ayr ON N0B1E0   
3   102 Stonewalk Dr Kemptville ON K0G 1J0        Kemptville ON K0G 1J0   
4    407 105 Dunbrack St Halifax NS B3M3G7            Halifax NS B3M3G7   

  effective_date                             municipality_district  \
0    Apr/11/2025                                          Kingston   
1    Apr/17/2025  Halifax Regional Municipality - West Chezzetcook   
2    May/01/2025    Township of North Dumfries, Region of Waterloo   
3    Apr/15/2025                          North Grenville, Ontario   
4    Apr/17/2025           Halifax Regional Municipality - Halifax   

                  site_dimensions   lot_size_sf units_sq_ft year_built  \
0           Condo Common Property           n/a       

In [6]:
comps = []

for a in data['appraisals']:
    for comp in a['comps']:
        comp['orderID'] = a['orderID']  # track origin
        comps.append(comp)

comps_df = pd.DataFrame(comps)
print(comps_df.head())

  distance_to_subject  prop_type   stories             address  \
0             0.15 KM  Townhouse  2 Storey  930 Amberdale Cres   
1             0.02 KM  Townhouse  2 Storey      771 Ashwood Dr   
2             0.09 KM  Townhouse  2 Storey  995 Amberdale Cres   
3             3.73 KM   Detached  1 Storey      64 Deermist Dr   
4             8.98 KM   Detached  1 Storey       85 Oceanic Dr   

                  city_province    sale_date sale_price     dom  \
0           Kingston ON K7M 6V1  Oct/25/2024    378,900      38   
1           Kingston ON K7M 6X7  Feb/05/2025    327,000      51   
2           Kingston ON K7M 6X6  Feb/14/2025    315,000      95   
3       Porters Lake NS B3E 1P3  Jan/16/2025    800,000  141+/-   
4  East Lawrencetown NS B2Z 1T6  Dec/21/2024    950,000  280+/-   

  location_similarity         lot_size    age condition          gla  \
0             Similar  N/A Condominium     49  Superior    1044 SqFt   
1            Inferior  N/A Condominium     49  Inferior 

In [7]:
properties = []

for a in data['appraisals']:
    for p in a['properties']:
        p['orderID'] = a['orderID']
        properties.append(p)

properties_df = pd.DataFrame(properties)
print(properties_df)


          id                     address  bedrooms     gla       city  \
0        367         463 Conservatory Dr       3.0  1500.0   Kingston   
1     163443     463 Conservatory Drive        3.0  1750.0  Kingston    
2        378              311 Janette St       3.0  1500.0   Kingston   
3     130023         311 Janette Street        3.0  1300.0  Kingston    
4       2782                4056 Bath Rd       4.0     NaN   Kingston   
...      ...                         ...       ...     ...        ...   
9815  282586             13 Aragon Road        3.0  1750.0  Kingston    
9816  288860      1029 Bluffwood Avenue        4.0  3250.0  Kingston    
9817  286281       491 Maple Lawn Drive        4.0  4250.0  Kingston    
9818  286938  1224 Greenwood Park Drive        4.0  2250.0  Kingston    
9819  288707      1018 Rainbow Crescent        3.0  1300.0  Kingston    

     province postal_code   property_sub_type                 structure_type  \
0     Ontario     K7M 9C8            Detach

In [9]:
import json
import pandas as pd

# Load the JSON file
with open('../data/raw_appraisals_dataset.json', 'r') as f:
    data = json.load(f)

rows = []

# Iterate over each appraisal
for appraisal in data['appraisals']:
    order_id = appraisal['orderID']
    subject = appraisal['subject']

    comps = appraisal.get('comps', [])
    properties = appraisal.get('properties', [])

    # Handle empty comps/properties
    if not comps:
        comps = [None]
    if not properties:
        properties = [None]

    # Combine each comp and property with the subject
    for comp in comps:
        for prop in properties:
            row = {
                'orderID': order_id,
                **{f"subject_{k}": v for k, v in subject.items()}
            }
            if comp:
                row.update({f"comp_{k}": v for k, v in comp.items()})
            if prop:
                row.update({f"property_{k}": v for k, v in prop.items()})

            rows.append(row)

# Convert to DataFrame
df = pd.DataFrame(rows)

print(df.head())

   orderID                          subject_address  \
0  4762597  142-950 Oakview Ave Kingston ON K7M 6W8   
1  4762597  142-950 Oakview Ave Kingston ON K7M 6W8   
2  4762597  142-950 Oakview Ave Kingston ON K7M 6W8   
3  4762597  142-950 Oakview Ave Kingston ON K7M 6W8   
4  4762597  142-950 Oakview Ave Kingston ON K7M 6W8   

  subject_subject_city_province_zip subject_effective_date  \
0                "Twin Oak Meadows"            Apr/11/2025   
1                "Twin Oak Meadows"            Apr/11/2025   
2                "Twin Oak Meadows"            Apr/11/2025   
3                "Twin Oak Meadows"            Apr/11/2025   
4                "Twin Oak Meadows"            Apr/11/2025   

  subject_municipality_district subject_site_dimensions subject_lot_size_sf  \
0                      Kingston   Condo Common Property                 n/a   
1                      Kingston   Condo Common Property                 n/a   
2                      Kingston   Condo Common Property    

In [ ]:
developer_prompt = '''

# Identity #

You are a data scientist with a focus on real estate and property appraisal. Your primary goal on this next project
is to clean up the dataset that you have been provided in order to streamline the data analysis process for ML and
AI toooling.



# Dataset Context #

* The dataset being provided is in the form of a JSON. It a list of appraisals containing 4 main components. The first
component is an orderID which is a unique identifier for each appraisal.

* For evert orderID there is a subject property. This is the property that is being appraised.

* Following the subject property is a list of comps. The comps component contains a list of three properties which were
determined to have characteristics and qualities that are most comparable to the subject property for the purpose of
appraising the subject property.

* The last component for every appraisal is a properties list. The properties list is an extensive list of properties
that are being compared to the subject property. It might be the case that the comp properties are not listed in the
properties list.



# Instructions #

* Your primary goal is to clean up and standardize the dataset so the same comparable fields can be used in order to
train a ML model to determine the best comps given the subject property and list of possibel comparable properties.

* The primary focus when dealing with the dataset is standardizing fields, and not so much on the data or the values
associated with the fields. For example, the lot_size_sf field might be provided as a float value, like 3345.3, but it
might also be provided as a string value like '3345 sqft'. In other cases, it might be provided as a string with
completely different units like '1.8 acres'.

* Your goal is to do limited computations and conversions in terms of standardizing fields. When provided values with
notable varying units or other anamolies, focus on ensuring the integrity of the value as opposed to carrying out a
calculation to standardize the dataset.

* When provided a value as a string

* Ensure that spelling and capitalization is consistent across the dataset. For example, if a style for a property is
denoted as a '2-Storey', we want to use that same spelling and capitalization across all matching values. If a property
has a style denoted as 'two-story' or a '2-story', we would want to standardize it to '2-Storey'.

* Addresses in the address field need to be paid especially close attention to since they will be treated as unique
identifiers. Some of the addresses provided are full addresses like '142-950 Oakview Ave Kingston ON K7M 6W8'. The addresses
should be formatted to only include the street address which consists of the house/apartmnet number and the street that
the property resides on. Street names should be abbreviated in a standard manner. Street names, if they are numbered
streets, should have the numbers written out as integers. In the previously provided example, the correct address would be
'142 Oakview Ave.'.

* The JSON format of the dataset should be maintained.



# Sample Provided Raw Data #

"appraisals": [
    {
      "orderID": "4762597",
      "subject": {
        "address": "142-950 Oakview Ave Kingston ON K7M 6W8",
        "subject_city_province_zip": "\"Twin Oak Meadows\"",
        "effective_date": "Apr/11/2025",
        "municipality_district": "Kingston",
        "site_dimensions": "Condo Common Property",
        "lot_size_sf": "n/a",
        "units_sq_ft": "SqFt",
        "year_built": "1976",
        "structure_type": "Townhouse",
        "roofing": "Asphalt Shingle",
        "effective_age": "25",
        "style": "2 Storey",
        "construction": "Wood Frame",
        "remaining_economic_life": "50",
        "windows": "Vinyl",
        "basement": "Full/Finished",
        "exterior_finish": "Brick Veneer, Vinyl Siding",
        "basement_area": "522",
        "foundation_walls": "Concrete",
        "flooring": "Hardwood, carpet, laminate, ceramic",
        "plumbing_lines": "Copper, PEX, ABS",
        "heating": "Forced Air",
        "fuel_type": "Natural Gas",
        "water_heater": "40 Gallon - Gas",
        "cooling": "Central Air",
        "room_count": "6",
        "num_beds": "3",
        "room_total": "6",
        "main_lvl_area": "522",
        "second_lvl_area": "522",
        "third_lvl_area": "",
        "gla": "1044",
        "subject_age": "49",
        "num_baths": "1:1",
        "condition": "Average"
      },
      "comps": [
        {
          "distance_to_subject": "0.15 KM",
          "prop_type": "Townhouse",
          "stories": "2 Storey",
          "address": "930 Amberdale Cres",
          "city_province": "Kingston ON K7M 6V1",
          "sale_date": "Oct/25/2024",
          "sale_price": "378,900",
          "dom": "38",
          "location_similarity": "Similar",
          "lot_size": "N/A Condominium",
          "age": "49",
          "condition": "Superior",
          "gla": "1044 SqFt",
          "room_count": "6",
          "bed_count": "3",
          "bath_count": "2:0",
          "basement_finish": "Full/Finished",
          "parking": "No Garage",
          "neighborhood": ""
        },
        {
          "distance_to_subject": "0.02 KM",
          "prop_type": "Townhouse",
          "stories": "2 Storey",
          "address": "771 Ashwood Dr",
          "city_province": "Kingston ON K7M 6X7",
          "sale_date": "Feb/05/2025",
          "sale_price": "327,000",
          "dom": "51",
          "location_similarity": "Inferior",
          "lot_size": "N/A Condominium",
          "age": "49",
          "condition": "Inferior",
          "gla": "1044 SqFt",
          "room_count": "6",
          "bed_count": "3",
          "bath_count": "1:0",
          "basement_finish": "Full/Finished",
          "parking": "No Garage",
          "neighborhood": ""
        },
        {
          "distance_to_subject": "0.09 KM",
          "prop_type": "Townhouse",
          "stories": "2 Storey",
          "address": "995 Amberdale Cres",
          "city_province": "Kingston ON K7M 6X6",
          "sale_date": "Feb/14/2025",
          "sale_price": "315,000",
          "dom": "95",
          "location_similarity": "Inferior",
          "lot_size": "N/A Condominium",
          "age": "49",
          "condition": "Inferior",
          "gla": "1044 SqFt",
          "room_count": "6",
          "bed_count": "3",
          "bath_count": "2:0",
          "basement_finish": "Full/Part Finished",
          "parking": "No Garage",
          "neighborhood": ""
        }
      ],
      "properties": [
        {
          "id": 367,
          "address": "463 Conservatory Dr",
          "bedrooms": 3,
          "gla": 1500,
          "city": "Kingston",
          "province": "Ontario",
          "postal_code": "K7M 9C8",
          "property_sub_type": "Detached",
          "structure_type": "Detached",
          "style": "Brick, Vinyl Siding",
          "levels": "Two",
          "room_count": 11,
          "full_baths": 3,
          "half_baths": null,
          "main_level_finished_area": null,
          "upper_lvl_fin_area": null,
          "bg_fin_area": null,
          "lot_size_sf": 3555.5,
          "year_built": null,
          "roof": null,
          "basement": "Unfinished",
          "cooling": "Central Air",
          "heating": "Forced Air",
          "close_price": 674000.0,
          "close_date": "2025-01-13",
          "public_remarks": "Welcome to this beautifully maintained and spacious family home! This charming residence features three generous bedrooms and two and a half bathrooms. Enjoy entertaining in the separate dining and living rooms, or relax in the open-concept kitchen that seamlessly flows into the inviting eat-in area and family room, complete with a cozy gas fireplace. Step outside through the sliding doors to discover a spacious deck and a lovely, secluded backyard perfect for family gatherings and outdoor enjoyment. Throughout the main and upper floors, you'll find elegant hardwood and ceramic flooring. The upper level boasts three large bedrooms, along with a bonus space ideal for a children's play area or additional living space. Enjoy the convenience of a full bathroom plus a four-piece ensuite. The property also includes a large garage and a double-wide driveway, providing ample parking for up to four vehicles. We can't wait to welcome you home!",
          "latitude": 44.2325,
          "longitude": -76.5901
        },
        {
          "id": 163443,
          "address": "463 Conservatory Drive ",
          "bedrooms": 3,
          "gla": 1750,
          "city": "Kingston ",
          "province": "Ontario",
          "postal_code": "K7M 9C8",
          "property_sub_type": "Detached",
          "structure_type": "Detached, 2-Storey ",
          "style": "2-Storey ",
          "levels": "2-Storey ",
          "room_count": 11,
          "full_baths": null,
          "half_baths": null,
          "main_level_finished_area": null,
          "upper_lvl_fin_area": null,
          "bg_fin_area": null,
          "lot_size_sf": 3535.0,
          "year_built": null,
          "roof": "",
          "basement": "Unfinished ",
          "cooling": "Central Air ",
          "heating": "Forced Air ",
          "close_price": 674000.0,
          "close_date": "2025-01-13",
          "public_remarks": "Welcome to this beautifully maintained and spacious family home! This charming residence features threegenerous bedrooms and two and a half bathrooms. Enjoy entertaining in the separate dining and livingrooms, or relax in the open-concept kitchen that seamlessly flows into the inviting eat-in area and familyroom, complete with a cozy gas fireplace.Step outside through the sliding doors to discover a spacious deck and a lovely, secludedbackyardperfect for family gatherings and outdoor enjoyment. Throughout the main and upper floors,youll find elegant hardwood and ceramic flooring.The upper level boasts three large bedrooms, along with a bonus space ideal for a children's play area oradditional living space. Enjoy the convenience of a full bathroom plus a four-piece ensuite. The propertyalso includes a large garage and a double-wide driveway, providing ample parking for up to four vehicles.We cant wait to welcome you home!",
          "latitude": 44.2325,
          "longitude": -76.5901
        },
        {
          "id": 378,
          "address": "311 Janette St",
          "bedrooms": 3,
          "gla": 1500,
          "city": "Kingston",
          "province": "Ontario",
          "postal_code": "K7P 0K8",
          "property_sub_type": "Freehold Townhouse",
          "structure_type": "Freehold Townhouse",
          "style": "2-Storey",
          "levels": "Two",
          "room_count": 11,
          "full_baths": 4,
          "half_baths": null,
          "main_level_finished_area": null,
          "upper_lvl_fin_area": null,
          "bg_fin_area": null,
          "lot_size_sf": 2622.0,
          "year_built": null,
          "roof": null,
          "basement": "Fin W/O",
          "cooling": "Central Air",
          "heating": "Forced Air",
          "close_price": 585000.0,
          "close_date": "2025-01-14",
          "public_remarks": "Come and see what this fully finished, Barr constructed, townhome has to offer. Sitting on a quiet crescent, on the edge of Woodhaven, is 311 Janette, an end unit townhome, with a deep lot; no feeling like you're in a fishbowl here! The main floor is an open space, with laminate floors, ample storage, lots of southern sunshine, a discrete powder room by the front door and a deck off the eating area - perfect for a BBQ. Upstairs offers hardwood flooring (2023) and three bedrooms, including a generous primary suite, two full bathrooms, and a convenient laundry setup. The basement is a fantastic, finished rec-room space with a walkout to your fenced (2023) yard! There is ample storage adjacent to the utility space and you'll love the useful cold storage area! This townhome has so much to offer we're sure you'll agree.",
          "latitude": 44.2622,
          "longitude": -76.5904
        },]]}


# Sample Response Data #

what fields must be kept:

- address
- prop_type
- stories
- city_province
- sale_date
- sale_price
- dom
- location_similarity
- lot_size
- age
- condition
- gla
- room_count
- bed_count
- bath_count
- basement_finish
- parking
- neighborhood

    {
      "orderID": "4762597",
      "subject": {
        "address": "142-950 Oakview Ave.",
        "city_province": "Kingston, Ontario ON K7M ",
        "prop_type": "Townhouse",
        "stories": "2-Storey",
        "gla": 1044,
        "location_similarity": "N/A",
        "lot_size": "N/A",
        "age": "49",
        "condition": "N/A",
        "room_count": "6",
        "bath_count": "1:1",
        "bed_count": "3",
        "basement_finish": "Full/Finished",
        "parking": "Garage",
        "neighborhood": "N/A",
        "roof": "Asphalt Shingle",
        "cooling": "Central Air",
        "heating": "Forced Air"
      },
      "comps": [
        {
          "distance_to_subject": "0.15 KM",
          "prop_type": "Townhouse",
          "stories": "2 Storey",
          "address": "930 Amberdale Cres.",
          "city_province": "Kingston ON K7M 6V1",
          "sale_date": "Oct/25/2024",
          "sale_price": "378,900",
          "dom": "38",
          "location_similarity": "Similar",
          "lot_size": "N/A Condominium",
          "age": "49",
          "condition": "Superior",
          "gla": "1044 SqFt",
          "room_count": "6",
          "bed_count": "3",
          "bath_count": "2:0",
          "basement_finish": "Full/Finished",
          "parking": "No Garage",
          "neighborhood": "",
          "roof": "N/A",
          "cooling": "N/A",
          "heating": "N/A"
        },
        {
          "distance_to_subject": "0.02 KM",
          "prop_type": "Townhouse",
          "stories": "2 Storey",
          "address": "771 Ashwood Dr.",
          "city_province": "Kingston, Ontario K7M 6X7",
          "sale_date": "Feb/05/2025",
          "sale_price": "327,000",
          "dom": "51",
          "location_similarity": "Inferior",
          "lot_size": "N/A Condominium",
          "age": "49",
          "condition": "Inferior",
          "gla": "1044 SqFt",
          "room_count": "6",
          "bed_count": "3",
          "bath_count": "1:0",
          "basement_finish": "Full/Finished",
          "parking": "No Garage",
          "neighborhood": "",
          "roof": "N/A",
          "cooling": "N/A",
          "heating": "N/A"
        },
        {
          "distance_to_subject": "0.09 KM",
          "prop_type": "Townhouse",
          "stories": "2 Storey",
          "address": "995 Amberdale Cres.",
          "city_province": "Kingston, Ontario K7M 6X6",
          "sale_date": "Feb/14/2025",
          "sale_price": "315,000",
          "dom": "95",
          "location_similarity": "Inferior",
          "lot_size": "N/A Condominium",
          "age": "49",
          "condition": "Inferior",
          "gla": "1044 SqFt",
          "room_count": "6",
          "bed_count": "3",
          "bath_count": "2:0",
          "basement_finish": "Full/Part Finished",
          "parking": "No Garage",
          "neighborhood": "",
          "roof": "N/A",
          "cooling": "N/A",
          "heating": "N/A"
        }
      ],
      "properties": [
        {
          "address": "463 Conservatory Dr.",
          "city_province": "Kingston, Ontario K7M 9C8",
          "prop_type": "Detached",
          "stories": "2-Storey",
          "gla": 1500,
          "sale_date": "2025-01-13",
          "sale_price": 674000.0,
          "dom": "N/A",
          "location_similarity": "N/A",
          "lot_size": 3555.5,
          "age": "N/A",
          "condition": "N/A",
          "room_count": "11",
          "bath_count": "2:1",
          "bed_count": "3",
          "basement_finish": "Unfinished",
          "parking": "Garage",
          "neighborhood": "N/A",
          "roof": "N/A",
          "cooling": "Central Air",
          "heating": "Forced Air",
          "latitude": 44.2325,
          "longitude": -76.5901
        },
        {
          "address": "311 Janette St.",
          "city_province": "Kingston, Ontario K7P 0K8",
          "prop_type": "Freehold Townhouse",
          "stories": "2-Storey",
          "gla": 1500,
          "sale_date": "2025-01-14",
          "sale_price": 585000.0,
          "dom": "N/A",
          "location_similarity": "N/A",
          "lot_size": 2622.0,
          "age": "N/A",
          "condition": "N/A",
          "room_count": "11",
          "bath_count": "3",
          "bed_count": "N/A",
          "basement_finish": "Finished Walk-Out",
          "parking": "N/A",
          "neighborhood": "N/A",
          "roof": "N/A",
          "cooling": "Central Air",
          "heating": "Forced Air",
          "latitude": 44.2622,
          "longitude": -76.5904
        }]

# Understanding Changes Made #

* The dataset was cleaned up. To begin, a lot of redundant fields were removed and cleaned up.

* A lot of the data that was redundant was removed and things were cleaned up overall

'''

In [ ]:
from openai import OpenAI
import json
import os
from dotenv import load_dotenv


with open('../data/raw_appraisals_dataset.json', 'r') as f:
    data = json.load(f)
    
load_dotenv()

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key= os.getenv("OPEN_ROUTER_API_KEY"),
)


data_appraisals = data["appraisals"]

# for appraisal in data_appraisals:
#   print(appraisal)

#   completion = client.chat.completions.create(
#   model="google/gemini-2.5-flash-preview",
#   messages=[
#     {
#       "role": "developer",
#       "content": developer_prompt
#     },
#     {
#         "role": "user",
#         "content": json.dumps(appraisal)
#     }
#   ]
#   )

#   print(completion.choices[0].message.content)

completion = client.chat.completions.create(
  model="google/gemini-2.5-flash-preview",
  messages=[
    {
      "role": "developer",
      "content": developer_prompt
    },
    {
        "role": "user",
        "content": json.dumps(data_appraisals[0])
    }
  ]
  )

print(completion.choices[0].message.content)

raw_response = completion.choices[0].message.content

# Assume `raw_response` is your string including \n and escape characters
cleaned = raw_response.strip("```json").strip("```").strip()


# Convert it from string to a Python dict
parsed_json = json.loads(cleaned)

# Write it to a .json file
with open("cleaned_appraisals.json", "w") as f:
    json.dump(parsed_json, f, indent=2)





```json
{
  "appraisals": [
    {
      "orderID": "4762597",
      "subject": {
        "address": "Oakview Ave.",
        "city_province": "Kingston, Ontario K7M",
        "prop_type": "Townhouse",
        "stories": "2-Storey",
        "gla": 1044,
        "sale_date": "Apr/11/2025",
        "sale_price": "N/A",
        "dom": "N/A",
        "location_similarity": "N/A",
        "lot_size": "N/A Condominium",
        "age": "49",
        "condition": "Average",
        "room_count": "6",
        "bath_count": "1:1",
        "bed_count": "3",
        "basement_finish": "Full/Finished",
        "parking": "No Garage",
        "neighborhood": "Twin Oak Meadows",
        "roof": "Asphalt Shingle",
        "cooling": "Central Air",
        "heating": "Forced Air"
      },
      "comps": [
        {
          "distance_to_subject": "0.15 KM",
          "prop_type": "Townhouse",
          "stories": "2-Storey",
          "address": "930 Amberdale Cres.",
          "city_province": "Kingsto

In [38]:
print(cleaned)

# Convert it from string to a Python dict
parsed_json = json.loads(cleaned)

# Write it to a .json file
with open("cleaned_appraisals.json", "w") as f:
    json.dump(parsed_json, f, indent=2)

{
  "appraisals": [
    {
      "orderID": "4762597",
      "subject": {
        "address": "Oakview Ave.",
        "city_province": "Kingston, Ontario K7M",
        "prop_type": "Townhouse",
        "stories": "2-Storey",
        "gla": 1044,
        "sale_date": "Apr/11/2025",
        "sale_price": "N/A",
        "dom": "N/A",
        "location_similarity": "N/A",
        "lot_size": "N/A Condominium",
        "age": "49",
        "condition": "Average",
        "room_count": "6",
        "bath_count": "1:1",
        "bed_count": "3",
        "basement_finish": "Full/Finished",
        "parking": "No Garage",
        "neighborhood": "Twin Oak Meadows",
        "roof": "Asphalt Shingle",
        "cooling": "Central Air",
        "heating": "Forced Air"
      },
      "comps": [
        {
          "distance_to_subject": "0.15 KM",
          "prop_type": "Townhouse",
          "stories": "2-Storey",
          "address": "930 Amberdale Cres.",
          "city_province": "Kingston, Ontar